In [1]:
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
import pandas as pd

clustered_vectors = pd.read_csv('./data/clustered_vectors.csv').values

qdrant = QdrantClient(":memory:")
collection_name = "vector_collection"

qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config={"size": clustered_vectors.shape[1], "distance": "Cosine"}
)

# Upload the vectors
points = [
    PointStruct(id=i, vector=vector.tolist())
    for i, vector in enumerate(clustered_vectors)
]

qdrant.upsert(collection_name=collection_name, points=points)

# source https://python-client.qdrant.tech/

C:\Users\dhimi\AppData\Local\Temp\ipykernel_35628\2033497685.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [2]:
# doing a search query test
query_vector = clustered_vectors[0].tolist()

search_result = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=10 
)

# Print the search results
for result in search_result:
    print(result)

id=0 version=0 score=0.9999999988241883 payload={} vector=None shard_key=None order_value=None
id=4791 version=0 score=0.9733456684059761 payload={} vector=None shard_key=None order_value=None
id=112 version=0 score=0.9731442626214278 payload={} vector=None shard_key=None order_value=None
id=596 version=0 score=0.9729512723259301 payload={} vector=None shard_key=None order_value=None
id=4477 version=0 score=0.97288293892145 payload={} vector=None shard_key=None order_value=None
id=6595 version=0 score=0.9728680538292809 payload={} vector=None shard_key=None order_value=None
id=4060 version=0 score=0.9728547648246987 payload={} vector=None shard_key=None order_value=None
id=9043 version=0 score=0.9728345841123753 payload={} vector=None shard_key=None order_value=None
id=1091 version=0 score=0.9728207069369069 payload={} vector=None shard_key=None order_value=None
id=2757 version=0 score=0.9727718166003076 payload={} vector=None shard_key=None order_value=None
